# 04 - Pandas - Processamento e Análise de Dados
## Aula 11 - 23 - Pandas - Remodelando Dataframes de Linhas Para Colunas (pivot_table e crosstab)

## Professor: Orlando Oliveira dos Santos, MsC.
 - E-mail: professor.orlando.santos@gmail.com 
 - Youtube :https://www.youtube.com/channel/UCPWWbkPWFmOjXH50TQWNAYg
 - Linkedin: https://linkedin.com/in/orlandoosantos
 - Facebook: https://www.facebook.com/proforlandosantosmsc/
 - Twitter: https://twitter.com/ProfOrlandoMsC
 - Instagram: https://www.instagram.com/proforlandosantosmsc/



## Formação Acadêmica
- Mestrado em Computação Aplicada - UnB (2014 – 2016)	
- MBA, Administração Estratégica de Sistemas de Informação – FGV (2011 – 2013)
- Tecnólogo em Processamento de Dados, Análise e Desenvolvimento de Sistemas – FAETEC/SP (2000-2002)

#  Pandas - Remodelagem e tabelas dinâmicas (pivot_table,crosstab)

In [1]:
import numpy as np
import pandas as pd
import os
import zipfile

diratual = os.getcwd()
print(diratual)

diretorio = os.path.join(diratual, 'content') 
diretorio


nome_arquivo_compactado = 'HIST_PAINEL_COVIDBR_18nov2020.zip'
with zipfile.ZipFile( os.path.join(diretorio, nome_arquivo_compactado), 'r') as zip_ref:
    zip_ref.extractall(diretorio)

C:\Orlando\Curso-de-Python-Para-Analise-de-Dados\04 - Pandas - Processamento e Análise de Dados


In [2]:
import pandas as pd
dfcovid = pd.read_csv(os.path.join(diretorio, 'HIST_PAINEL_COVIDBR_18nov2020.csv') , sep=';')
dfcovid.head()

,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-25,9,210147125.0,0,0,0,0,NaN,NaN,NaN
1,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-26,9,210147125.0,1,1,0,0,NaN,NaN,NaN
2,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-27,9,210147125.0,1,0,0,0,NaN,NaN,NaN
3,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-28,9,210147125.0,1,0,0,0,NaN,NaN,NaN
4,Brasil,NaN,NaN,76,NaN,NaN,NaN,2020-02-29,9,210147125.0,2,1,0,0,NaN,NaN,NaN


In [3]:
filtro = (dfcovid['estado'].isna() == False ) & ( dfcovid['codmun'].isna()  )
dfcovid = dfcovid[filtro][['regiao', 'estado', 'data' , 'populacaoTCU2019',
       'casosNovos', 'obitosNovos']]

dfcovid['mes'] = dfcovid['data'].map(lambda x : x[:7])


print(dfcovid.shape)
dfcovid.head()

(7290, 7)


,regiao,estado,data,populacaoTCU2019,casosNovos,obitosNovos,mes
270,Norte,RO,2020-02-25,1777225.0,0,0,2020-02
271,Norte,RO,2020-02-26,1777225.0,0,0,2020-02
272,Norte,RO,2020-02-27,1777225.0,0,0,2020-02
273,Norte,RO,2020-02-28,1777225.0,0,0,2020-02
274,Norte,RO,2020-02-29,1777225.0,0,0,2020-02


In [4]:
dfregiaouf = dfcovid[['regiao', 'estado', 'mes',  'casosNovos', 'obitosNovos', 'populacaoTCU2019', 'data']]\
  .groupby(['regiao', 'estado', 'mes', 'data' ])\
  .agg({'casosNovos':'sum', 'obitosNovos':'sum', 'populacaoTCU2019':'max'})\
  .reset_index()
dfregiaouf.head()

,regiao,estado,mes,data,casosNovos,obitosNovos,populacaoTCU2019
0,Centro-Oeste,DF,2020-02,2020-02-25,0,0,3015268.0
1,Centro-Oeste,DF,2020-02,2020-02-26,0,0,3015268.0
2,Centro-Oeste,DF,2020-02,2020-02-27,0,0,3015268.0
3,Centro-Oeste,DF,2020-02,2020-02-28,0,0,3015268.0
4,Centro-Oeste,DF,2020-02,2020-02-29,0,0,3015268.0


In [5]:
dfregiaouf.columns

Index(['regiao', 'estado', 'mes', 'data', 'casosNovos', 'obitosNovos',
       'populacaoTCU2019'],
      dtype='object')

# Remodelagem e tabelas dinâmicas 

## Tabelas dinâmicas 
O Pandas possui dois métodos para gerar tabelas dinâmicas:
- pivot() fornece pivotamento de propósito geral com vários tipos de dados (strings, numéricos, etc.) 
- pivot_table() fornece pivotamento com agregação de dados numéricos.

## pivot_table() 
A função pivot_table()pode ser usada para criar tabelas dinâmicas no estilo planilha. 
- Utiliza os seguintes argumentos:

 - data: um objeto DataFrame.

 - values: uma coluna ou uma lista de colunas para agregar.

 - index: uma coluna, Grouper, array que tem o mesmo comprimento dos dados, ou uma lista deles. Chaves para agrupar no índice da tabela dinâmica. Se um array é passado, ele está sendo usado da mesma maneira que os valores das colunas.

 - columns: uma coluna, Grouper, array que tem o mesmo comprimento dos dados, ou uma lista deles. Chaves para agrupar na coluna da tabela dinâmica. Se um array é passado, ele está sendo usado da mesma maneira que os valores das colunas.

 - aggfunc: função a ser usada para agregação, assumindo como **padrão numpy.mean**

In [6]:
dfresumo = pd.pivot_table(dfregiaouf, values=['casosNovos', 'obitosNovos'], 
               index=['mes'], columns=['regiao', 'estado'])
dfresumo.head(20)

casosNovos                                                    \
regiao  Centro-Oeste                                          Nordeste   
estado            DF           GO          MS           MT          AL   
mes                                                                      
2020-02     0.000000     0.000000    0.000000     0.000000    0.000000   
2020-03    10.709677     2.096774    1.548387     0.806452    0.580645   
2020-04    34.133333    23.866667    6.900000     9.066667   34.200000   
2020-05   271.741935    94.225806   39.806452    68.774194  298.193548   
2020-06  1314.600000   689.300000  215.866667   440.233333  855.800000   
2020-07  1841.096774  1405.161290  547.451613  1168.677419  766.548387   
2020-08  1798.387097  2082.741935  774.225806  1273.580645  615.451613   
2020-09  1006.766667  2563.500000  692.300000  1069.000000  269.166667   
2020-10   677.419355  1485.032258  407.677419   628.580645  126.129032   
2020-11   479.550000   852.350000  421.050000   544.900000  122.800000   

                                                                          ...  \
regiao                                                                    ...   
estado            BA           CE           MA           PB           PE  ...   
mes                                                                       ...   
2020-02     0.000000     0.000000     0.000000     0.000000     0.000000  ...   
2020-03     6.870968    12.580645     1.000000     0.548387     2.806452  ...   
2020-04    87.933333   240.533333   105.300000    26.566667   226.300000  ...   
2020-05   501.322581  1318.806452  1014.483871   398.322581   889.483871  ...   
2020-06  1830.500000  2007.000000  1527.066667  1126.500000   813.600000  ...   
2020-07  2995.064516  2102.677419  1297.096774  1156.032258  1166.032258  ...   
2020-08  2921.709677  1324.870968   998.516129   741.419355   984.967742  ...   
2020-09  1793.300000   873.533333   731.633333   516.300000   721.066667  ...   
2020-10  1375.193548  1065.354839   395.580645   383.290323   507.161290  ...   
2020-11  1450.350000   787.000000   256.300000   423.300000   570.950000  ...   

        obitosNovos                                                        \
regiao        Norte                        Sudeste                          
estado           RO        RR        TO         ES         MG          RJ   
mes                                                                         
2020-02    0.000000  0.000000  0.000000   0.000000   0.000000    0.000000   
2020-03    0.032258  0.000000  0.000000   0.000000   0.064516    0.741935   
2020-04    0.500000  0.233333  0.100000   2.766667   2.666667   27.700000   
2020-05    4.516129  3.516129  2.258065  16.806452   6.096774  144.838710   
2020-06   12.066667  5.566667  4.233333  34.800000  23.133333  157.866667   
2020-07   11.419355  7.161290  5.838710  28.935484  58.193548  109.580645   
2020-08    8.903226  2.645161  9.419355  19.774194  82.774194   83.483871   
2020-09    6.966667  2.033333  8.866667  12.900000  67.500000   80.733333   
2020-10    3.225806  1.419355  5.096774  10.000000  53.387097   68.161290   
2020-11    2.900000  1.400000  2.500000  12.300000  33.650000   66.900000   

                                                      
regiao                     Sul                        
estado           SP         PR         RS         SC  
mes                                                   
2020-02    0.000000   0.000000   0.000000   0.000000  
2020-03    4.387097   0.096774   0.129032   0.064516  
2020-04   74.633333   2.666667   1.566667   1.466667  
2020-05  169.032258   3.193548   5.580645   3.129032  
2020-06  238.266667  15.566667  13.000000   6.600000  
2020-07  265.612903  41.000000  40.709677  24.548387  
2020-08  226.354839  44.193548  50.290323  37.354839  
2020-09  186.933333  39.833333  44.900000  17.900000  
2020-10  119.000000  23.580645  32.741935  10.225806  
2020-11   93.400000  30.800000  32.500000  1

In [7]:
dfresumo = pd.pivot_table(dfregiaouf, values=['casosNovos', 'obitosNovos'], 
               index=['regiao', 'estado'], columns=['mes'])
dfresumo.head(20)

casosNovos                                      \
mes                    2020-02    2020-03     2020-04      2020-05   
regiao       estado                                                  
Centro-Oeste DF            0.0  10.709677   34.133333   271.741935   
             GO            0.0   2.096774   23.866667    94.225806   
             MS            0.0   1.548387    6.900000    39.806452   
             MT            0.0   0.806452    9.066667    68.774194   
Nordeste     AL            0.0   0.580645   34.200000   298.193548   
             BA            0.0   6.870968   87.933333   501.322581   
             CE            0.0  12.580645  240.533333  1318.806452   
             MA            0.0   1.000000  105.300000  1014.483871   
             PB            0.0   0.548387   26.566667   398.322581   
             PE            0.0   2.806452  226.300000   889.483871   
             PI            0.0   0.580645   16.500000   142.516129   
             RN            0.0   2.645161   36.500000   200.806452   
             SE            0.0   0.612903   14.266667   211.354839   
Norte        AC            0.0   1.354839   12.066667   187.580645   
             AM            0.0   5.645161  169.300000  1165.290323   
             AP            0.0   0.322581   35.666667   274.903226   
             PA            0.0   1.032258   94.800000  1131.774194   
             RO            0.0   0.258065   16.466667   143.225806   
             RR            0.0   0.516129   16.766667    95.322581   
             TO            0.0   0.354839    4.200000   130.290323   

                                                                         \
mes                      2020-06      2020-07      2020-08      2020-09   
regiao       estado                                                       
Centro-Oeste DF      1314.600000  1841.096774  1798.387097  1006.766667   
             GO       689.300000  1405.161290  2082.741935  2563.500000   
             MS       215.866667   547.451613   774.225806   692.300000   
             MT       440.233333  1168.677419  1273.580645  1069.000000   
Nordeste     AL       855.800000   766.548387   615.451613   269.166667   
             BA      1830.500000  2995.064516  2921.709677  1793.300000   
             CE      2007.000000  2102.677419  1324.870968   873.533333   
             MA      1527.066667  1297.096774   998.516129   731.633333   
             PB      1126.500000  1156.032258   741.419355   516.300000   
             PE       813.600000  1166.032258   984.967742   721.066667   
             PI       516.366667  1001.774194   836.387097   622.700000   
             RN       762.833333   649.322581   365.548387   256.166667   
             SE       613.733333  1074.258065   445.645161   163.733333   
Norte        AC       234.466667   205.548387   162.000000   119.166667   
             AM       981.500000   971.516129   624.387097   634.333333   
             AP       629.666667   257.290323   216.935484   168.700000   
             PA      2174.833333  1660.612903  1447.451613  1029.766667   
             RO       543.633333   572.290323   521.322581   358.600000   
             RR       372.233333   560.483871   371.032258   231.033333   
             TO       219.600000   453.548387   834.516129   576.966667   

                                          obitosNovos                       \
mes                      2020-10  2020-11     2020-02   2020-03    2020-04   
regiao       estado                                                          
Centro-Oeste DF       677.419355   479.55         0.0  0.096774   0.900000   
             GO      1485.032258   852.35         0.0  0.032258   0.933333   
             MS       407.677419   421.05         0.0  0.032258   0.266667   
             MT       628.580645   544.90         0.0  0.000000   0.366667   
Nordeste     AL       126.129032   122.80         0.0  0.032258   1.533333   
             BA      1375.193548  1450.35         0.0  0.06

In [8]:
dfresumo = pd.pivot_table(dfregiaouf, values=['casosNovos', 'obitosNovos'], 
               index=['mes'], columns=['regiao', 'estado'], 
               aggfunc=np.sum)
dfresumo.head(20)

casosNovos                                                     \
regiao  Centro-Oeste                      Nordeste                        
estado            DF     GO     MS     MT       AL     BA     CE     MA   
mes                                                                       
2020-02            0      0      0      0        0      0      0      0   
2020-03          332     65     48     25       18    213    390     31   
2020-04         1024    716    207    272     1026   2638   7216   3159   
2020-05         8424   2921   1234   2132     9244  15541  40883  31449   
2020-06        39438  20679   6476  13207    25674  54915  60210  45812   
2020-07        57074  43560  16971  36229    23763  92847  65183  40210   
2020-08        55750  64565  24001  39481    19079  90573  41071  30954   
2020-09        30203  76905  20769  32070     8075  53799  26206  21949   
2020-10        21000  46036  12638  19486     3910  42631  33026  12263   
2020-11         9591  17047   8421  10898     2456  29007  15740   5126   

                       ... obitosNovos                                      \
regiao                 ...       Norte           Sudeste                     
estado      PB     PE  ...          RO   RR   TO      ES    MG    RJ    SP   
mes                    ...                                                   
2020-02      0      0  ...           0    0    0       0     0     0     0   
2020-03     17     87  ...           1    0    0       0     2    23   136   
2020-04    797   6789  ...          15    7    3      83    80   831  2239   
2020-05  12348  27574  ...         140  109   70     521   189  4490  5240   
2020-06  33795  24408  ...         362  167  127    1044   694  4736  7148   
2020-07  35837  36147  ...         354  222  181     897  1804  3397  8234   
2020-08  22984  30534  ...         276   82  292     613  2566  2588  7017   
2020-09  15489  21632  ...         209   61  266     387  2025  2422  5608   
2020-10  11882  15722  ...         100   44  158     310  1655  2113  3689   
2020-11   8466  11419  ...          58   28   50     246   673  1338  1868   

                           
regiao    Sul              
estado     PR    RS    SC  
mes                        
2020-02     0     0     0  
2020-03     3     4     2  
2020-04    80    47    44  
2020-05    99   173    97  
2020-06   467   390   198  
2020-07  1271  1262   761  
2020-08  1370  1559  1158  
2020-09  1195  1347   537  
2020-10   731  1015   317  
2020-11   616   650   321  

[10 rows x 54 columns]

In [9]:
dfresumo = pd.pivot_table(dfregiaouf, values=['casosNovos', 'obitosNovos'], 
               index=['regiao', 'estado'], columns=['mes'],
                aggfunc=np.sum)
dfresumo.head(20)

casosNovos                                          \
mes                    2020-02 2020-03 2020-04 2020-05 2020-06 2020-07   
regiao       estado                                                      
Centro-Oeste DF              0     332    1024    8424   39438   57074   
             GO              0      65     716    2921   20679   43560   
             MS              0      48     207    1234    6476   16971   
             MT              0      25     272    2132   13207   36229   
Nordeste     AL              0      18    1026    9244   25674   23763   
             BA              0     213    2638   15541   54915   92847   
             CE              0     390    7216   40883   60210   65183   
             MA              0      31    3159   31449   45812   40210   
             PB              0      17     797   12348   33795   35837   
             PE              0      87    6789   27574   24408   36147   
             PI              0      18     495    4418   15491   31055   
             RN              0      82    1095    6225   22885   20129   
             SE              0      19     428    6552   18412   33302   
Norte        AC              0      42     362    5815    7034    6372   
             AM              0     175    5079   36124   29445   30117   
             AP              0      10    1070    8522   18890    7976   
             PA              0      32    2844   35085   65245   51479   
             RO              0       8     494    4440   16309   17741   
             RR              0      16     503    2955   11167   17375   
             TO              0      11     126    4039    6588   14060   

                                                    obitosNovos          \
mes                 2020-08 2020-09 2020-10 2020-11     2020-02 2020-03   
regiao       estado                                                       
Centro-Oeste DF       55750   30203   21000    9591           0       3   
             GO       64565   76905   46036   17047           0       1   
             MS       24001   20769   12638    8421           0       1   
             MT       39481   32070   19486   10898           0       0   
Nordeste     AL       19079    8075    3910    2456           0       1   
             BA       90573   53799   42631   29007           0       2   
             CE       41071   26206   33026   15740           0       7   
             MA       30954   21949   12263    5126           0       1   
             PB       22984   15489   11882    8466           0       0   
             PE       30534   21632   15722   11419           0       6   
             PI       25928   18681   17205    8716           0       4   
             RN       11332    7685   11743    3689           0       1   
             SE       13815    4912    6886    2521           0       0   
Norte        AC        5022    3575    2574    3450           0       0   
             AM       19356   19030   22357   11175           0       3   
             AP        6725    5061    3921    3910           0       0   
             PA       44871   30893   22470   11891           0       0   
             RO       16161   10758    5755    5063           0       1   
             RR       11502    6931    6835    4269           0       0   
             TO       25870   17309    7566    3589           0       0   

                                                                             \
mes                 2020-04 2020-05 2020-06 2020-07 2020-08 2020-09 2020-10   
regiao       estado                                                           
Centro-Oeste DF          27     140     417     882    1052     734     428   
             GO          28      95     350    1182    1438    1578    1081   
             MS           8      11      56     300     486     441     289   
             MT          11      50     532    1226     932     655     392   
Nordeste     AL          46     396   

### O pivot_tabletambém está disponível como um método de instância no DataFrame, ie DataFrame.pivot_table().

In [10]:
dfresumo = dfregiaouf.pivot_table(values=['casosNovos', 'obitosNovos'], 
               index=['regiao', 'estado'], columns=['mes'],
                aggfunc=np.sum)
dfresumo.head(20)

casosNovos                                          \
mes                    2020-02 2020-03 2020-04 2020-05 2020-06 2020-07   
regiao       estado                                                      
Centro-Oeste DF              0     332    1024    8424   39438   57074   
             GO              0      65     716    2921   20679   43560   
             MS              0      48     207    1234    6476   16971   
             MT              0      25     272    2132   13207   36229   
Nordeste     AL              0      18    1026    9244   25674   23763   
             BA              0     213    2638   15541   54915   92847   
             CE              0     390    7216   40883   60210   65183   
             MA              0      31    3159   31449   45812   40210   
             PB              0      17     797   12348   33795   35837   
             PE              0      87    6789   27574   24408   36147   
             PI              0      18     495    4418   15491   31055   
             RN              0      82    1095    6225   22885   20129   
             SE              0      19     428    6552   18412   33302   
Norte        AC              0      42     362    5815    7034    6372   
             AM              0     175    5079   36124   29445   30117   
             AP              0      10    1070    8522   18890    7976   
             PA              0      32    2844   35085   65245   51479   
             RO              0       8     494    4440   16309   17741   
             RR              0      16     503    2955   11167   17375   
             TO              0      11     126    4039    6588   14060   

                                                    obitosNovos          \
mes                 2020-08 2020-09 2020-10 2020-11     2020-02 2020-03   
regiao       estado                                                       
Centro-Oeste DF       55750   30203   21000    9591           0       3   
             GO       64565   76905   46036   17047           0       1   
             MS       24001   20769   12638    8421           0       1   
             MT       39481   32070   19486   10898           0       0   
Nordeste     AL       19079    8075    3910    2456           0       1   
             BA       90573   53799   42631   29007           0       2   
             CE       41071   26206   33026   15740           0       7   
             MA       30954   21949   12263    5126           0       1   
             PB       22984   15489   11882    8466           0       0   
             PE       30534   21632   15722   11419           0       6   
             PI       25928   18681   17205    8716           0       4   
             RN       11332    7685   11743    3689           0       1   
             SE       13815    4912    6886    2521           0       0   
Norte        AC        5022    3575    2574    3450           0       0   
             AM       19356   19030   22357   11175           0       3   
             AP        6725    5061    3921    3910           0       0   
             PA       44871   30893   22470   11891           0       0   
             RO       16161   10758    5755    5063           0       1   
             RR       11502    6931    6835    4269           0       0   
             TO       25870   17309    7566    3589           0       0   

                                                                             \
mes                 2020-04 2020-05 2020-06 2020-07 2020-08 2020-09 2020-10   
regiao       estado                                                           
Centro-Oeste DF          27     140     417     882    1052     734     428   
             GO          28      95     350    1182    1438    1578    1081   
             MS           8      11      56     300     486     441     289   
             MT          11      50     532    1226     932     655     392   
Nordeste     AL          46     396   

## Adicionando margens (Totais)
Se você passar margins=True para pivot_table, Allcolunas e linhas especiais serão adicionadas com agregações parciais de grupos nas categorias nas linhas e colunas:

In [11]:
dfresumo = dfregiaouf.pivot_table(values=['obitosNovos'], 
               index=['regiao', 'estado'], columns=['mes'],
                aggfunc=np.sum,
                margins=True, margins_name = 'Total')
dfresumo.head(28)

obitosNovos                                          \
mes                     2020-02 2020-03 2020-04 2020-05 2020-06 2020-07   
regiao       estado                                                       
Centro-Oeste DF               0       3      27     140     417     882   
             GO               0       1      28      95     350    1182   
             MS               0       1       8      11      56     300   
             MT               0       0      11      50     532    1226   
Nordeste     AL               0       1      46     396     609     515   
             BA               0       2     102     563    1186    1610   
             CE               0       7     475    2528    3136    1522   
             MA               0       1     183     771    1093     965   
             PB               0       0      62     298     617     834   
             PE               0       6     559    2242    2022    1728   
             PI               0       4      20     137     502     666   
             RN               0       1      55     249     729     743   
             SE               0       0      12     146     518     758   
Norte        AC               0       0      19     129     217     166   
             AM               0       3     422    1627     771     445   
             AP               0       0      34     188     195     148   
             PA               0       0     208    2715    1997     808   
             RO               0       1      15     140     362     354   
             RR               0       0       7     109     167     222   
             TO               0       0       3      70     127     181   
Sudeste      ES               0       0      83     521    1044     897   
             MG               0       2      80     189     694    1804   
             RJ               0      23     831    4490    4736    3397   
             SP               0     136    2239    5240    7148    8234   
Sul          PR               0       3      80      99     467    1271   
             RS               0       4      47     173     390    1262   
             SC               0       2      44      97     198     761   
Total                         0     201    5700   23413   30280   32881   

                                                             
mes                 2020-08 2020-09 2020-10 2020-11   Total  
regiao       estado                                          
Centro-Oeste DF        1052     734     428     174    3857  
             GO        1438    1578    1081     448    6201  
             MS         486     441     289     121    1713  
             MT         932     655     392     227    4025  
Nordeste     AL         320     185     166      73    2311  
             BA        1934    1347     878     439    8061  
             CE         741     585     359     124    9477  
             MA         433     310     297     165    4218  
             PB         639     374     277     135    3236  
             PE        1036     658     376     272    8899  
             PI         496     302     273     168    2568  
             RN         479     137     184      80    2657  
             SE         423     179     170      68    2274  
Norte        AC          81      47      34      18     711  
             AM         381     507     387     218    4761  
             AP          96      48      39      39     787  
             PA         418     427     170     112    6855  
             RO         276     209     100      58    1515  
             RR          82      61      44      28     720  
             TO         292     266     158      50    1147  
Sudeste      ES         613     387     310     246    4101  
             MG        2566    2025    1655     673    9688  
             RJ        2588    2422    2113    1338   21938  
             SP        7017    5608    3689    1868   41179  
Sul          PR     

## Tabulações cruzadas 


- crosstab() é usado para calcular uma tabulação cruzada de dois (ou mais) fatores. 
- Por padrão, crosstabcalcula uma tabela de frequência dos fatores
- Se pode fazer outros calculos se uma matriz de valores e uma função de agregação for passadas.

Esse método aceita vários parâmetros

- index: tipo array, valores para agrupar nas linhas.

- columns: tipo array, valores para agrupar nas colunas.

- values: tipo array, opcional, array de valores para agregar de acordo com os fatores.

- aggfunc: função, opcional, se nenhum array de valores for passado, calcula uma tabela de frequência.

- rownames: sequência, padrão None, deve corresponder ao número de arrays de linha passadas.

- colnames: sequência, padrão None, se passado, deve corresponder ao número de arrays de coluna passadas.

- margins: booleano, padrão False, Adicionar margens de linha / coluna (subtotais)

- normalize: booleano, {'all', 'index', 'columns'} ou {0,1}, padrão False. Normalize dividindo todos os valores pela soma dos valores.

In [12]:
dfresumo = pd.crosstab(dfregiaouf['mes'], [dfregiaouf['regiao'], dfregiaouf['estado']], 
                       rownames=['Meses'], colnames=['Região', 'Estados'])
dfresumo.head(28)

Região  Centro-Oeste             Nordeste                      ... Norte      \
Estados           DF  GO  MS  MT       AL  BA  CE  MA  PB  PE  ...    RO  RR   
Meses                                                          ...             
2020-02            5   5   5   5        5   5   5   5   5   5  ...     5   5   
2020-03           31  31  31  31       31  31  31  31  31  31  ...    31  31   
2020-04           30  30  30  30       30  30  30  30  30  30  ...    30  30   
2020-05           31  31  31  31       31  31  31  31  31  31  ...    31  31   
2020-06           30  30  30  30       30  30  30  30  30  30  ...    30  30   
2020-07           31  31  31  31       31  31  31  31  31  31  ...    31  31   
2020-08           31  31  31  31       31  31  31  31  31  31  ...    31  31   
2020-09           30  30  30  30       30  30  30  30  30  30  ...    30  30   
2020-10           31  31  31  31       31  31  31  31  31  31  ...    31  31   
2020-11           20  20  20  20       20  20  20  20  20  20  ...    20  20   

Região      Sudeste             Sul          
Estados  TO      ES  MG  RJ  SP  PR  RS  SC  
Meses                                        
2020-02   5       5   5   5   5   5   5   5  
2020-03  31      31  31  31  31  31  31  31  
2020-04  30      30  30  30  30  30  30  30  
2020-05  31      31  31  31  31  31  31  31  
2020-06  30      30  30  30  30  30  30  30  
2020-07  31      31  31  31  31  31  31  31  
2020-08  31      31  31  31  31  31  31  31  
2020-09  30      30  30  30  30  30  30  30  
2020-10  31      31  31  31  31  31  31  31  
2020-11  20      20  20  20  20  20  20  20  

[10 rows x 27 columns]

In [13]:
dfresumo = pd.crosstab(dfregiaouf['mes'], [dfregiaouf['regiao'], dfregiaouf['estado']], 
                       rownames=['Meses'], colnames=['Região', 'Estados'],
                       values=dfregiaouf['obitosNovos'], aggfunc=np.sum)
dfresumo.head(28)

Região  Centro-Oeste                  Nordeste                               \
Estados           DF    GO   MS    MT       AL    BA    CE    MA   PB    PE   
Meses                                                                         
2020-02            0     0    0     0        0     0     0     0    0     0   
2020-03            3     1    1     0        1     2     7     1    0     6   
2020-04           27    28    8    11       46   102   475   183   62   559   
2020-05          140    95   11    50      396   563  2528   771  298  2242   
2020-06          417   350   56   532      609  1186  3136  1093  617  2022   
2020-07          882  1182  300  1226      515  1610  1522   965  834  1728   
2020-08         1052  1438  486   932      320  1934   741   433  639  1036   
2020-09          734  1578  441   655      185  1347   585   310  374   658   
2020-10          428  1081  289   392      166   878   359   297  277   376   
2020-11          174   448  121   227       73   439   124   165  135   272   

Região   ... Norte           Sudeste                     Sul              
Estados  ...    RO   RR   TO      ES    MG    RJ    SP    PR    RS    SC  
Meses    ...                                                              
2020-02  ...     0    0    0       0     0     0     0     0     0     0  
2020-03  ...     1    0    0       0     2    23   136     3     4     2  
2020-04  ...    15    7    3      83    80   831  2239    80    47    44  
2020-05  ...   140  109   70     521   189  4490  5240    99   173    97  
2020-06  ...   362  167  127    1044   694  4736  7148   467   390   198  
2020-07  ...   354  222  181     897  1804  3397  8234  1271  1262   761  
2020-08  ...   276   82  292     613  2566  2588  7017  1370  1559  1158  
2020-09  ...   209   61  266     387  2025  2422  5608  1195  1347   537  
2020-10  ...   100   44  158     310  1655  2113  3689   731  1015   317  
2020-11  ...    58   28   50     246   673  1338  1868   616   650   321  

[10 rows x 27 columns]

Se crosstab  receber apenas duas Séries, resultará uma tabela de frequência.

In [14]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'])
dfresumo.head(28)

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,20,45,35,20,15
2020-03,124,279,217,124,93
2020-04,120,270,210,120,90
2020-05,124,279,217,124,93
2020-06,120,270,210,120,90
2020-07,124,279,217,124,93
2020-08,124,279,217,124,93
2020-09,120,270,210,120,90
2020-10,124,279,217,124,93


In [15]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'], values=dfregiaouf['obitosNovos'], aggfunc=np.sum
                       , margins=True)
dfresumo.head(28)

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul,All
mes,,,,,,
2020-02,0,0,0,0,0,0
2020-03,5,22,4,161,9,201
2020-04,74,1514,708,3233,171,5700
2020-05,296,7330,4978,10440,369,23413
2020-06,1355,10412,3836,13622,1055,30280
2020-07,3590,9341,2324,14332,3294,32881
2020-08,3908,6501,1626,12784,4087,28906
2020-09,3408,4077,1565,10442,3079,22571
2020-10,2190,2980,932,7767,2063,15932


### Normalização dos dados
As tabelas de frequência também podem ser normalizadas para mostrar porcentagens em vez de contagens usando o argumento normalize

In [16]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize=True,
                      values=dfregiaouf['obitosNovos'], aggfunc=np.sum)
print(dfresumo.sum())
print(dfresumo.sum().sum())
dfresumo.head(28)

regiao
Centro-Oeste    0.093682
Nordeste        0.259179
Norte           0.097834
Sudeste         0.456110
Sul             0.093196
dtype: float64
1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.000030,0.000130,0.000024,0.000955,0.000053
2020-04,0.000439,0.008979,0.004199,0.019174,0.001014
2020-05,0.001755,0.043472,0.029523,0.061917,0.002188
2020-06,0.008036,0.061751,0.022750,0.080789,0.006257
2020-07,0.021291,0.055399,0.013783,0.084999,0.019536
2020-08,0.023177,0.038556,0.009643,0.075819,0.024239
2020-09,0.020212,0.024180,0.009282,0.061929,0.018261
2020-10,0.012988,0.017674,0.005527,0.046064,0.012235


normalize também pode normalizar valores em cada linha ou em cada coluna:

In [17]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='columns',
                      values=dfregiaouf['obitosNovos'], aggfunc=np.sum)
print(dfresumo.sum())
print(dfresumo.sum().sum())
dfresumo.head(28)

regiao
Centro-Oeste    1.0
Nordeste        1.0
Norte           1.0
Sudeste         1.0
Sul             1.0
dtype: float64
5.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.000317,0.000503,0.000242,0.002093,0.000573
2020-04,0.004685,0.034645,0.042919,0.042038,0.010882
2020-05,0.018739,0.167731,0.301770,0.135750,0.023482
2020-06,0.085781,0.238255,0.232541,0.177125,0.067138
2020-07,0.227273,0.213748,0.140883,0.186357,0.209622
2020-08,0.247404,0.148761,0.098569,0.166229,0.260087
2020-09,0.215751,0.093293,0.094871,0.135776,0.195940
2020-10,0.138643,0.068191,0.056499,0.100993,0.131284


In [18]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='index',
                      values=dfregiaouf['obitosNovos'], aggfunc=np.sum)
print(dfresumo.sum(1))
print(dfresumo.sum(1).sum())
dfresumo.head(28)

mes
2020-02    0.0
2020-03    1.0
2020-04    1.0
2020-05    1.0
2020-06    1.0
2020-07    1.0
2020-08    1.0
2020-09    1.0
2020-10    1.0
2020-11    1.0
dtype: float64
9.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.024876,0.109453,0.019900,0.800995,0.044776
2020-04,0.012982,0.265614,0.124211,0.567193,0.030000
2020-05,0.012643,0.313074,0.212617,0.445906,0.015760
2020-06,0.044749,0.343857,0.126684,0.449868,0.034841
2020-07,0.109182,0.284085,0.070679,0.435875,0.100179
2020-08,0.135197,0.224901,0.056251,0.442261,0.141389
2020-09,0.150990,0.180630,0.069337,0.462629,0.136414
2020-10,0.137459,0.187045,0.058499,0.487509,0.129488


In [19]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='all',
                      values=dfregiaouf['obitosNovos'], aggfunc=np.sum)
print(dfresumo.sum())
print(dfresumo.sum().sum())
print(dfresumo.sum(1))
print(dfresumo.sum(1).sum())
dfresumo.head(28)

regiao
Centro-Oeste    0.093682
Nordeste        0.259179
Norte           0.097834
Sudeste         0.456110
Sul             0.093196
dtype: float64
1.0
mes
2020-02    0.000000
2020-03    0.001192
2020-04    0.033805
2020-05    0.138856
2020-06    0.179583
2020-07    0.195009
2020-08    0.171434
2020-09    0.133863
2020-10    0.094489
2020-11    0.051769
dtype: float64
1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.000030,0.000130,0.000024,0.000955,0.000053
2020-04,0.000439,0.008979,0.004199,0.019174,0.001014
2020-05,0.001755,0.043472,0.029523,0.061917,0.002188
2020-06,0.008036,0.061751,0.022750,0.080789,0.006257
2020-07,0.021291,0.055399,0.013783,0.084999,0.019536
2020-08,0.023177,0.038556,0.009643,0.075819,0.024239
2020-09,0.020212,0.024180,0.009282,0.061929,0.018261
2020-10,0.012988,0.017674,0.005527,0.046064,0.012235


### Adicionando margens totaol
Finalmente, também se pode adicionar margens ou normalizar esta saída.

In [20]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='all',
                      values=dfregiaouf['obitosNovos'], aggfunc=np.sum,
                      margins=True, margins_name = 'Total')
dfresumo.head(28)

regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul,Total
mes,,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.000030,0.000130,0.000024,0.000955,0.000053,0.001192
2020-04,0.000439,0.008979,0.004199,0.019174,0.001014,0.033805
2020-05,0.001755,0.043472,0.029523,0.061917,0.002188,0.138856
2020-06,0.008036,0.061751,0.022750,0.080789,0.006257,0.179583
2020-07,0.021291,0.055399,0.013783,0.084999,0.019536,0.195009
2020-08,0.023177,0.038556,0.009643,0.075819,0.024239,0.171434
2020-09,0.020212,0.024180,0.009282,0.061929,0.018261,0.133863
2020-10,0.012988,0.017674,0.005527,0.046064,0.012235,0.094489


In [21]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='columns',
                      values=dfregiaouf['obitosNovos'], aggfunc=np.sum,
                      margins=True, margins_name = 'Total')
print(dfresumo.Total.sum())
dfresumo.head(30)

1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul,Total
mes,,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.000317,0.000503,0.000242,0.002093,0.000573,0.001192
2020-04,0.004685,0.034645,0.042919,0.042038,0.010882,0.033805
2020-05,0.018739,0.167731,0.301770,0.135750,0.023482,0.138856
2020-06,0.085781,0.238255,0.232541,0.177125,0.067138,0.179583
2020-07,0.227273,0.213748,0.140883,0.186357,0.209622,0.195009
2020-08,0.247404,0.148761,0.098569,0.166229,0.260087,0.171434
2020-09,0.215751,0.093293,0.094871,0.135776,0.195940,0.133863
2020-10,0.138643,0.068191,0.056499,0.100993,0.131284,0.094489


In [22]:
dfresumo.index

Index(['2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07',
       '2020-08', '2020-09', '2020-10', '2020-11'],
      dtype='object', name='mes')

In [23]:
dfresumo = pd.crosstab(dfregiaouf['mes'], dfregiaouf['regiao'] , normalize='index',
                      values=dfregiaouf['obitosNovos'], aggfunc=np.sum,
                      margins=True, margins_name = 'Total')
print(dfresumo.loc['Total'].sum())
dfresumo.head(30)

1.0


regiao,Centro-Oeste,Nordeste,Norte,Sudeste,Sul
mes,,,,,
2020-02,0.000000,0.000000,0.000000,0.000000,0.000000
2020-03,0.024876,0.109453,0.019900,0.800995,0.044776
2020-04,0.012982,0.265614,0.124211,0.567193,0.030000
2020-05,0.012643,0.313074,0.212617,0.445906,0.015760
2020-06,0.044749,0.343857,0.126684,0.449868,0.034841
2020-07,0.109182,0.284085,0.070679,0.435875,0.100179
2020-08,0.135197,0.224901,0.056251,0.442261,0.141389
2020-09,0.150990,0.180630,0.069337,0.462629,0.136414
2020-10,0.137459,0.187045,0.058499,0.487509,0.129488
